
<table>
  <tr>
    <td> <center><img src="images/header1.png" width=400></center> </td>
  </tr>
</table>

<h1><center>Основы машинного обучения</center></h1>
<hr>
<h2><center>Методы обучения без учителя: Методы понижения размерности</center></h2>
<h3><center>Лектор: Ефимов Владислав</center></h3>
<h3><center>Лекции готовили: Шестаков Андрей, Ефимов Владислав</center></h3>

In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy

plt.style.use('seaborn-talk')
plt.rcParams['figure.figsize'] = (12,8)

import warnings
warnings.filterwarnings('ignore')

# Для кириллицы на графиках
font = {'family': 'Verdana',
        'weight': 'normal'}
plt.rc('font', **font)

try:
    from ipywidgets import interact, IntSlider, fixed, FloatSlider
except ImportError:
    print(u'Так надо')
    
from IPython.display import YouTubeVideo

# Методы понижения размерности признаков

## Для чего можно понижать размерность признаков?

* Cмотреть на 2-3 признака удобнее чем на 100
* Потенциально может улучшить качество моделей
* Удаление лишних коррелирующих признаков
* Ускоряет обучение
* Данные занимают меньше места
* Меньше признаков - легче интерпретировать модели

## Способы понижения размерности

Избавляться от размерности можно методами **отбора признаков (Feature Selection)** и методами **уменьшения размерности (Feature Reduction)**

**Глобальная разница**
* Feature Selection: не используем часть признаков
* Feature Reduction: исходные признаки проходят через некоторое преобразование $f(\cdot)$, и на выходе признаков становится меньше

# Principal Component Analysis
## Метод Главных Компонент

## Интерпретация

* Интерпретация 1: находит такие ортогональные оси, вдоль которых дисперсия данных максимальна

<center><img src='http://www.visiondummy.com/wp-content/uploads/2014/05/correlated_2d.png' width=400></center>

<center><img src='images/pca.png' width=800></center>

## Интерпретация

* Интерпретация 2: Найти такое подпространство меньшей размерности $L$ Такое что различие между точками и их проекциями минимальна

<center><img src='images/plane_best_fit.png' width=800></center>

<center><img src='images/pca_example.png' width=800></center>

## В двух словах:

* Совершаем переход к новым осям, так что:
    * Новые переменные являются линейной комбинацией старых переменных
    * Новые оси ортогональны друг другу
    * Дисперсия вдоль новых осей максимальная

## Что значит "перейти к новым координатам"?

* Пусть у нас есть объект $x$ с тремя признаками: $x=(-0.343, -0.754, 0.241)$
* Можно сказать, что он представлен в пространстве, натянутом на 3 стандартных базистых вектора:
$$ e_1 = \left( \begin{array}{c}
1 \\
0 \\
0
\end{array} \right) \quad
e_2 = \left( \begin{array}{c}
0 \\
1 \\
0
\end{array} \right) \quad
e_3 = \left( \begin{array}{c}
0 \\
0 \\
1
\end{array} \right) \quad$$

$$ x = -0.343 e_1 + -0.754 e_2 + 0.241 e_3 $$

* Предположими мы хотим перейти к другому базису, например:

$$ w_1 = \left( \begin{array}{c}
-0.390 \\
0.089 \\
-0.916
\end{array} \right) \quad
w_2 = \left( \begin{array}{c}
-0.639 \\
-0.742 \\
0.200
\end{array} \right) \quad
w_3 = \left( \begin{array}{c}
-0.663 \\
0.664 \\
0.346
\end{array} \right) \quad$$
* Как спроицировать наш объект $x$ из исходного базиса в данный?


## Проецируем

$$ w_1 = \left( \begin{array}{c}
-0.390 \\
0.089 \\
-0.916
\end{array} \right) \quad
w_2 = \left( \begin{array}{c}
-0.639 \\
-0.742 \\
0.200
\end{array} \right) \quad
w_3 = \left( \begin{array}{c}
-0.663 \\
0.664 \\
0.346
\end{array} \right) \quad$$


$$ z = W^\top x = \left( \begin{array}{ccc}
-0.390 & 0.089 & -0.916\\
-0.639 & -0.742 & 0.200 \\
-0.663 & 0.664 & 0.346
\end{array} \right)
\left( \begin{array}{c}
-0.343 \\
-0.754 \\
0.241
\end{array} \right) = 
\left( \begin{array}{c}
-1.154 \\
0.828 \\
0.190
\end{array} \right)$$

То есть: $$ x = -1.154 w_1 + 0.828 w_2 + 0.190 w_3$$

(Пример взят из [Mohammed J. Zaki, Ch7](https://www.amazon.com/Data-Mining-Analysis-Fundamental-Algorithms/dp/0521766338) )

* Проецировать научились, осталось только понять как же находить эти самые $w$?
    * Новые оси ортогональны друг другу
    * Дисперсия вдоль новых осей максимальна

# Находим главные компоненты
* Новые оси ортогональны друг другу:
    $$\langle w_{i},w_{j}\rangle=\begin{cases}
1, & i=j\\
0 & i\ne j
\end{cases}$$

* Дисперсия вдоль новых осей максимальна
    * Предполагаем, что исходные признаки **центрированы**
$$
\begin{align} \sigma^2_w & = \frac{1}{n-1}\sum\limits_{i=1}^n(z_i - \mu_z)^2 \\
& = \frac{1}{n-1}\sum\limits_{i=1}^n(w^\top x_i)^2 \\
& = \frac{1}{n-1}\sum\limits_{i=1}^n w^\top( x_i x_i^\top) w \\
& = w^\top \left(\frac{1}{n-1}\sum\limits_{i=1}^n x_i x_i^\top \right) w \\
& = w^\top C w \rightarrow \\
& = w^\top \frac{1}{n-1} X^\top X w \rightarrow \max_w \\
\end{align}
$$
* $C = \frac{1}{n-1} X^\top X$ - ковариационная матрица

* [Interpreting Covariance Matrix](http://www.visiondummy.com/2014/04/geometric-interpretation-covariance-matrix/)

# Находим главные компоненты
То есть, чтобы найти первую главную компоненту, надо решить такую задачу:
$$
\begin{equation}
\begin{cases}
w_1^\top C w_1  \rightarrow \max_{w_1} \\
w_1^\top w_1 = 1
\end{cases}
\end{equation}
$$

## Решение для первой компоненты

Изначально было так 
$$ w_1^\top C w_1  \rightarrow \max_{w_1}$$

* Строим лагранжиан
$$ \mathcal{L}(w_1, \nu) = w_1^\top C w_1 - \nu (w_1^\top w_1 - 1) \rightarrow max_{w_1, \nu}$$
* Считем производную по $w_1$
$$ \frac{\partial\mathcal{L}}{\partial w_1} = C w_1 - \nu w_1 = 0 \Leftrightarrow X^\top X w_1 = \nu w_1$$

Подставим одно в другое:
$$ w_1^\top C w_1 = \nu w_1^\top w_1 = \nu \rightarrow \max$$

## Задача о собственном векторе, собственном числе матрицы

Оказывается, что у матрицы ковариций $C$:
* Все собственные числа $\lambda_i \in \mathbb{R}, \lambda_i \geq 0$ (упорядочим индексы по возрастанию так, что $\lambda_1 \geq \lambda_2  \geq \dots \geq \lambda_d $)
* Собственные вектора при $\lambda_i \neq \lambda_j $ ортогональны: $v_i^\top v_j = 0$
* Для уникальных $\lambda_i$ уникальны и $v_i$

## Решение для первой компоненты

Изначально было так 
$$ w_1^\top C w_1  \rightarrow \max_{w_1}$$

* Строим лагранжиан
$$ \mathcal{L}(w_1, \nu) = w_1^\top C w_1 - \nu (w_1^\top w_1 - 1) \rightarrow max_{w_1, \nu}$$
* Считем производную по $w_1$
$$ \frac{\partial\mathcal{L}}{\partial w_1} = C w_1 - \nu w_1 = 0 \Leftrightarrow С w_1 = \nu w_1$$

Подставим одно в другое:
$$ w_1^\top C w_1 = \nu w_1^\top w_1 = \nu \rightarrow \max$$

Что значит, что:
* $\nu$ - наибольшее собственное число матрицы $C$, а именно - $\lambda_1$
* $w_1$ - собственный вектор при $\lambda_1$

## Задача и решение для второй компоненты

$$
\begin{equation}
\begin{cases}
w_2^\top C w_2  \rightarrow \max_{w_2} \\
w_2^\top w_2 = 1 \\
w_2^\top w_1 = 0
\end{cases}
\end{equation}
$$

Аналогичными операциями мы прийдем к тому, что  $w_2$ - собственный вектор при втором по величине собственном числе матрицы $C$. 

То есть поиск главных компонент сводится к поиску собственных векторов и собственных чисел матрицы $C$!


## Алгоритм

1. Центрируем и _шкалируем_ исходные признаки

    1.1. Вообще шкалирование для самого алгоритма не обязательно, как мы видели из выкладок выше. Применяя шкалирование мы изначально "придаем" данным форму единичной сферы.
2. Считаем матрицу ковариаций $\frac{1}{n-1}X^\top X$
3. Находим $k$ главных компонент и собственных чисел
$$W = 
\left[
  \begin{array}{cccc}
    \mid & \mid & & \mid\\
    w_{1} & w_{2} & \ldots & w_{k} \\
    \mid & \mid & & \mid 
  \end{array}
\right]
$$
4. Проецируем на них:
$$ Z = XW $$ 

## Сколько компонент брать или зачем нужны $\lambda$?

* Из предыдущих слайдов мы помним, что 
$$w_1^\top C w_1 = \lambda_1$$
То есть $\lambda_1$ равна дисперсии точек, спроецированных на первую базовую компоненту
* Величина
$$
\frac{\lambda_{i}}{\sum_{d=1}^{D}\lambda_{d}}
$$
Показывает долю объясненной дисперии компоненты $i$

<center><img src='images/cumul_rat.png' width=900></center>

## Связь PCA и SVD

1. Пусть все также X центрировано, $C = \frac{1}{n-1} X^\top X$ -- ковариационная матрица.
2. Так как $C$ симметричная матрица, то она может быть представлена в виде $C = V L V^\top$, где $V$ это матрица с собственными векторами $C$, а $L$ диагональня матрица с собственными значениями по убыванию.
3. Сингулярное разложение позволяет нам представить произвольную матрицу в виде $X = U S V^\top$, где $U, V$ унитарные матрицы, $S$ диагональная матрица с сингулярными значениями по убыванию.
4. Подставим теперь разложение $X$, полученное через SVD, в определение $C$:
$$
C = \frac{1}{n-1} V S U^\top U S V^\top = V \frac{S^2}{n-1} V^\top
$$
5. Теперь осталось только сравнить это выражение для $C$ с ее представлением через собственные вектора. Легко видеть, что $V$ это и есть собственные вектора матрицы ковариаций, а собственные ее значения связаны с сингулярными значениями как $\lambda_i = s_i^2/(n-1)$.

## MNIST, DIGITS
<center><img src='images/digits.png' width=600></center>

## MNIST PCA

<center><img src='./images/mnist_pca.jpg'></center>

## Резюме

* PCA понижает размерность признакового пространства
* Новые компоненты являются линейной комбинацией исходных признаков
* Новые компоненты (не признаки) - ортогональны
* Можно применять в моделях и для визуализации
* [Tutorial 1](http://www.visiondummy.com/2014/05/feature-extraction-using-pca/)
* [PCA SVD туториал](https://www.cs.princeton.edu/picasso/mats/PCA-Tutorial-Intuition_jp.pdf)
* [Относительно легкая и подробная статья про связь PCA, SVD и связь между ними](https://jonathan-hui.medium.com/machine-learning-singular-value-decomposition-svd-principal-component-analysis-pca-1d45e885e491)

# Многомерное шкалирование

## Идея

* Перейти в пространство меньшей размерности так, чтобы расстояния между объектами в новом пространстве были подобны расстояниям в исходном пространстве.
* Дано $X = [x_1,\dots, x_n]\in \mathbb{R}^{N \times D}$ и/или $\delta_{ij}$ - мера близости между $(x_i,x_j)$
* Надо найти $Y = [y_1,\dots,y_n] \in \mathbb{R}^{N \times d}$ такие, что $\delta_{ij} \approx d(y_i, y_j) = \|y_i-y_j\|^2$

<center><img src='images/mds.png' width =1200></center>

Понятно, что точно воспроизвести расстояния получится не всегда

<center><img src='images/sphere_example.png'></center>

### Подходы
* Классический (cMDS)
* Метрический (metric MDS)
* Неметрический (non-metric MDS)

# t-SNE
## t-distributed stochastic neighbor embedding

* t-SNE - практически многомерное шкалирование
* Вместо этого мы будем пытаться перенести "окрестность" точек из исходного пространства в пространоство меньшей размерности
* Полученные расстояния скорее всего не будут соотносится с исходными

* Схожесть между объектами в исходном пространстве $\mathbb{R}^m$
$$
p(i, j) = \frac{p(i | j) + p(j | i)}{2n}, \quad p(j | i) = \frac{\exp(-\|\mathbf{x}_j-\mathbf{x}_i\|^2/{2 \sigma_i^2})}{\sum_{k \neq i}\exp(-\|\mathbf{x}_k-\mathbf{x}_i\|^2/{2 \sigma_i^2})}
$$
$\sigma_i$ неявно задается пользователем
* Схожесть между объектами в целевом пространстве $\mathbb{R}^k, k << m$
$$
q(i, j) = \frac{g(|\mathbf{y}_i - \mathbf{y}_j|)}{\sum_{k \neq l} g(|\mathbf{y}_i - \mathbf{y}_j|)}
$$ 
где $g(z) = \frac{1}{1 + z^2}$ - распределение Коши (t-распределение Стьюдента с 1 степенью свободы)
* Критерий
$$
J_{t-SNE}(y) = KL(P \| Q) = \sum_i \sum_j p(i, j) \log \frac{p(i, j)}{q(i, j)} \rightarrow \min\limits_{\mathbf{y}}
$$

## Дивергенция Кульбака-Лейблера

* Насколько распределение $P$ отличается от распределения $Q$?
$$
KL(P \| Q) = \sum_z P(z) \log \frac{P(z)}{Q(z)}
$$

<center><img src='images/kld.png' width=1200></center>

## Оптимизация

* Оптимизируем $J_{t-SNE}(y)$ с помощью градиентного спуска

$$\frac{\partial J_{t-SNE}}{\partial y_i}=4 \sum_j(p(i,j)−q(i,j))(y_i−y_j)g(|y_i−y_j|)$$

* [Статья](http://jmlr.csail.mit.edu/papers/volume9/vandermaaten08a/vandermaaten08a.pdf)
* [Примеры](http://lvdmaaten.github.io/tsne/)
* [Демо и советы](http://distill.pub/2016/misread-tsne/)
    * t-SNE может быть нестабильным
    * Размеры полученных сгустков могут ничего не значить
    * Расстояния между кластерами могут ничего не значить
    * Полностью шумовые данные могут выдать структуру

<center><img src='http://lvdmaaten.github.io/tsne/examples/mnist_tsne.jpg' width=600></center>

<center><img src='images/mainfold.png'></center>

# UMAP
## Uniform Manifold Approximation and Projection for Dimension Reduction

* Развитие подхода t-SNE.
* Более быстрый за счет отсутствия нормализации для исходного и нового пространств.
* Старается сохранить не только локальную, но и глобальную структуру в данных.

* Схожесть между объектами в исходном пространстве $\mathbb{R}^m$
$$
p_{ij} = p(i, j) = p(i | j) + p(j | i) - p(i | j)  p(j | i), \quad p(i | j) = \exp(-\frac{d(x_i, x_j) - \rho_i}{\sigma_i})
$$
$d(x_i, x_j)$ -- расстояние между точками (не обязательно евклидово), $\rho_i$ расстояние от $i$-го элемента до ближайшего соседа, $\sigma_i$ ищутся исходя из уравнения ($k$ -- гиперпараметр, число соседей):
$$\sum_j p(i,j) = \log_2 k$$
* Схожесть между объектами в целевом пространстве $\mathbb{R}^k, k < m$
$$
q_{ij} = q(i, j) = (1 + a(y_i - y_j)^{2b})^{-1}
$$ 
где $a\approx1.93$ и $b\approx0.79$ гиперпараметры и их значения по-умолчанию.
* Критерий -- fuzzy set cross entropy, кросс-энтропия для нечетких множеств
$$
CE(X, Y) = \sum_i \sum_j \left[ p_{ij}(X)\log\frac{p_{ij}(X)}{q_{ij}(Y)} - (1 - p_{ij}(X))\log\frac{(1-p_{ij}(X))}{(1-q_{ij}(Y))} \right]
$$
* Оптимизируемся с помощью стохастического градиентного спуска.

* [Оригинальная статья](https://arxiv.org/abs/1802.03426)
* [Статья с сравнением t-SNE и UMAP, а также более подробный разбор](https://towardsdatascience.com/how-exactly-umap-works-13e3040e1668)



<center><img src='images/umap_vs_other.png' width=700></center>

# Литература
* [Mohammed J. Zaki, Ch7](https://www.amazon.com/Data-Mining-Analysis-Fundamental-Algorithms/dp/0521766338)
* [Мodern MDS](http://www.springer.com/la/book/9780387251509)
* [Habrahabr](https://habrahabr.ru/post/267041/)

## Вопросы?

### Пожалуйста, напишите отзыв о лекции